In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
j_faults = pd.read_csv('../data/J1939Faults.csv').reset_index(drop = True)
onboard_data = pd.read_csv('../data/VehicleDiagnosticOnboardData.csv').reset_index(drop = True)
fault_codes = pd.read_excel('../data/Service Fault Codes_1_0_0_167.xlsx').reset_index(drop = True)
vehicle = pd.read_excel('../data/Vehicle_Make.xlsx').reset_index(drop = True)

C:\Users\hjrko\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
ob_data = onboard_data.pivot(index='FaultId', columns='Name',values = 'Value').rename_axis(None)
pd.set_option('display.max_columns', None)
ob_data.reset_index(inplace=True)
ob_data = ob_data.rename(columns = {'index':'FaultId'})

In [4]:
ob_faults = pd.merge(j_faults, ob_data, how='left', left_on='RecordID', right_on='FaultId')

In [5]:
ob_faults['null_count'] = ob_faults.isnull().sum(axis=1)

In [ ]:
problem_cols = ['AcceleratorPedal',
 'BarometricPressure',
 'CruiseControlActive',
 'CruiseControlSetSpeed',
 'DistanceLtd',
 'EngineCoolantTemperature',
 'EngineLoad',
 'EngineOilPressure',
 'EngineOilTemperature',
 'EngineRpm',
 'EngineTimeLtd',
 'FuelLevel',
 'FuelLtd',
 'FuelRate',
 'FuelTemperature',
 'IgnStatus',
 'IntakeManifoldTemperature',
 'LampStatus',
 'ParkingBrake',
 'ServiceDistance',
 'Speed',
 'SwitchedBatteryVoltage',
 'Throttle',
 'TurboBoostPressure']

for x in problem_cols:
    try:
        ob_faults[x] = ob_faults[x].str.replace(',','').astype(float)
    except ValueError:
        ob_faults[x] = ob_faults[x]
ob_faults['EventTimeStamp'] =  pd.to_datetime(ob_faults['EventTimeStamp'], format='%Y%m%d %H:%M:%S.%f')
ob_faults['LocationTimeStamp'] =  pd.to_datetime(ob_faults['EventTimeStamp'], format='%Y%m%d %H:%M:%S.%f')

In [ ]:
ob_faults['repair_center'] = False
ob_faults.loc[(ob_faults['Longitude'].astype('str').str.contains(r'-86.43\d+')) & (ob_faults['Latitude'].astype('str').str.contains(r'36.06\d+')), 'repair_center'] = True
ob_faults.loc[(ob_faults['Longitude'].astype('str').str.contains(r'-86.44\d+')) & (ob_faults['Latitude'].astype('str').str.contains(r'35.58\d+')), 'repair_center'] = True
ob_faults.loc[(ob_faults['Longitude'].astype('str').str.contains(r'-83.17\d+')) & (ob_faults['Latitude'].astype('str').str.contains(r'36.19\d+')), 'repair_center'] = True
ob_faults = ob_faults[ob_faults['repair_center']==False]

In [ ]:
ob_faults['EquipmentID'] = ob_faults['EquipmentID'].astype(str)
ob_faults = ob_faults[ob_faults['EquipmentID'].str.len()  < 6]

In [ ]:
combined_df = ob_faults.drop(columns=['actionDescription','repair_center','faultValue'])

In [ ]:
combined_df

In [ ]:
condensed_df = combined_df.sort_values(['EquipmentID', 'EventTimeStamp', 'spn', 'fmi'])[[
    'EquipmentID', 
    'EventTimeStamp', 
    'eventDescription', 
    'spn', 
    'fmi', 
    'active', 
    'activeTransitionCount', 
    'AcceleratorPedal',
    'BarometricPressure',
    'CruiseControlActive',
    'CruiseControlSetSpeed',
    'DistanceLtd',
    'EngineCoolantTemperature',
    'EngineLoad',
    'EngineOilPressure',
    'EngineOilTemperature',
    'EngineRpm',
    'EngineTimeLtd',
    'FuelLevel',
    'FuelLtd',
    'FuelRate',
    'FuelTemperature',
    'IgnStatus',
    'IntakeManifoldTemperature',
    'LampStatus',
    'ParkingBrake',
    'ServiceDistance',
    'Speed',
    'SwitchedBatteryVoltage',
    'Throttle',
    'TurboBoostPressure']]

mask = ((condensed_df['spn'].shift(-1) == 5246))

spn_prior_to_derate = condensed_df[mask]
spn_prior_to_derate = spn_prior_to_derate[~(spn_prior_to_derate['spn'] == 5246)]

spn_count = spn_prior_to_derate.groupby('spn').count().reset_index().sort_values('EquipmentID', ascending=False).head(20)

In [ ]:
spn_prior_to_derate

In [ ]:
problem_cols_num = [
    'AcceleratorPedal',
    'BarometricPressure',
    'CruiseControlSetSpeed',
    'DistanceLtd',
    'EngineCoolantTemperature',
    'EngineLoad',
    'EngineOilPressure',
    'EngineOilTemperature',
    'EngineRpm',
    'EngineTimeLtd',
    'FuelLevel',
    'FuelLtd',
    'FuelRate',
    'FuelTemperature',
    'IntakeManifoldTemperature',
    'LampStatus',
    'ServiceDistance',
    'Speed',
    'SwitchedBatteryVoltage',
    'Throttle',
    'TurboBoostPressure']

for x in problem_cols_num:
    spn_prior_to_derate.hist(x)
    plt.xlabel('Value')
    plt.ylabel('Bin Count')
    plt.title(x)

In [ ]:
fontsize = 40

fig, ax = plt.subplots(figsize=(60, 32))
sns.barplot(x = 'spn', 
            y = 'EquipmentID', 
            data = spn_count, 
            dodge = False, 
            palette = "deep",
            ax = ax,
            order = spn_count.sort_values('EquipmentID', ascending=False).spn)

plt.xlabel('SPN Code', fontsize = fontsize)
plt.xticks(fontsize = fontsize - 2)

plt.ylabel('Occurence Count', fontsize = fontsize)
plt.yticks(fontsize = fontsize - 2)

plt.title('Count oF SPNs that Occur Prior to a Full Derate', fontsize = fontsize);